In [1]:

import os, sys
import nest_asyncio
import asyncio
from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
import httpx

# 🧩 本地导入路径，根据放置的项目结构调整
sys.path.insert(0, r"E:\Github\a2a-samples\samples\python")

from components.api_key_dialog import api_key_dialog
from components.page_scaffold import page_scaffold
from pages.agent_list import agent_list_page
from pages.conversation import conversation_page
from pages.event_list import event_list_page
from pages.home import home_page_content
from pages.settings import settings_page_content
from pages.task_list import task_list_page
from service.server.server import ConversationServer
from state import host_agent_service
from state.state import AppState
import mesop as me
from contextlib import asynccontextmanager
from dotenv import load_dotenv

# ✅ 允许在 Notebook 中多次运行 asyncio
nest_asyncio.apply()

# ✅ 加载 .env 环境变量
load_dotenv()

# ⚙️ 初始化 FastAPI & 客户端封装
class HTTPXClientWrapper:
    async_client: httpx.AsyncClient = None

    def start(self):
        self.async_client = httpx.AsyncClient(timeout=30)

    async def stop(self):
        await self.async_client.aclose()
        self.async_client = None

    def __call__(self):
        assert self.async_client is not None
        return self.async_client


httpx_client_wrapper = HTTPXClientWrapper()

# ✅ 页面注册函数（照搬原代码结构）
def on_load(e: me.LoadEvent):
    state = me.state(AppState)
    me.set_theme_mode(state.theme_mode)
    if 'conversation_id' in me.query_params:
        state.current_conversation_id = me.query_params['conversation_id']
    else:
        state.current_conversation_id = ''
    uses_vertex_ai = os.getenv('GOOGLE_GENAI_USE_VERTEXAI', '').upper() == 'TRUE'
    api_key = os.getenv('GOOGLE_API_KEY', '')
    if uses_vertex_ai:
        state.uses_vertex_ai = True
    elif api_key:
        state.api_key = api_key
    else:
        state.api_key_dialog_open = True

security_policy = me.SecurityPolicy(allowed_script_srcs=['https://cdn.jsdelivr.net'])

@me.page(path='/', title='Chat', on_load=on_load, security_policy=security_policy)
def home_page():
    api_key_dialog()
    with page_scaffold():
        home_page_content(me.state(AppState))

@me.page(path='/agents', title='Agents', on_load=on_load, security_policy=security_policy)
def agents_page():
    api_key_dialog()
    agent_list_page(me.state(AppState))

@me.page(path='/conversation', title='Conversation', on_load=on_load, security_policy=security_policy)
def conversation():
    api_key_dialog()
    conversation_page(me.state(AppState))

@me.page(path='/event_list', title='Event List', on_load=on_load, security_policy=security_policy)
def events():
    api_key_dialog()
    event_list_page(me.state(AppState))

@me.page(path='/task_list', title='Task List', on_load=on_load, security_policy=security_policy)
def tasks():
    api_key_dialog()
    task_list_page(me.state(AppState))

@me.page(path='/settings', title='Settings', on_load=on_load, security_policy=security_policy)
def settings():
    api_key_dialog()
    settings_page_content()

# ✅ 定义 lifespan 生命周期（FastAPI 启动钩子）
@asynccontextmanager
async def lifespan(app: FastAPI):
    httpx_client_wrapper.start()
    ConversationServer(app, httpx_client_wrapper())
    app.openapi_schema = None
    app.mount("/", WSGIMiddleware(me.create_wsgi_app(debug_mode=False)))
    app.setup()
    yield
    await httpx_client_wrapper.stop()


## 异步任务状态更新

In [ ]:
import asyncio, uuid, threading
from IPython.display import display, clear_output



# 1. 已有漫长的同步函数
def run_pipeline_sync(user_input: str):
    import time
    time.sleep(5)                      
    return f"✅ 已处理: {user_input}"

# 2. 把同步函数丢进线程池 -> 变异步
async def run_pipeline_async(user_input: str):
    return await asyncio.to_thread(run_pipeline_sync, user_input)

# 3. ===== 以下是异步任务框架 =====
TASK_STATUS, TASK_RESULT = {}, {}

async def _long_task(task_id: str, user_input: str):
    TASK_STATUS[task_id] = "Running"
    try:
        result = await run_pipeline_async(user_input)
        TASK_RESULT[task_id] = result
        TASK_STATUS[task_id] = "Success"
    except Exception as e:
        TASK_STATUS[task_id] = f"Failed: {e}"

def start_task(user_input: str):
    task_id = str(uuid.uuid4())
    TASK_STATUS[task_id] = "Pending"
    asyncio.create_task(_long_task(task_id, user_input))
    return task_id

def get_task_status(task_id: str):
    return TASK_STATUS.get(task_id, "Unknown Task")

async def run_and_display_task(user_input: str):
    task_id = start_task(user_input)
    while True:
        status = get_task_status(task_id)
        clear_output(wait=True)
        display(f"任务 {task_id[:8]}… 状态：{status}")
        if status == "Success" or status.startswith("Failed"):
            break
        await asyncio.sleep(1)
    display("任务结果：", TASK_RESULT.get(task_id, "（无结果）"))


In [4]:
async def demo_multi():
    ids = [start_task(f"MSG {i}") for i in range(5)]
    while True:
        clear_output(wait=True)
        for tid in ids:
            print(f"{tid[:8]}  →  {get_task_status(tid)}")
        if all(get_task_status(tid).startswith(("Success", "Failed")) for tid in ids):
            break
        await asyncio.sleep(1)
    print("\n全部完成！结果预览：")
    for tid in ids:
        print(tid[:8], "=>", TASK_RESULT.get(tid))

await demo_multi()



87217666  →  Success
60fbc27a  →  Success
10ef5c1a  →  Success
e63244c8  →  Success
768644d8  →  Success

全部完成！结果预览：
87217666 => ✅ 已处理: MSG 0
60fbc27a => ✅ 已处理: MSG 1
10ef5c1a => ✅ 已处理: MSG 2
e63244c8 => ✅ 已处理: MSG 3
768644d8 => ✅ 已处理: MSG 4


## 服务启动

In [5]:
import uvicorn
from fastapi import FastAPI

app = FastAPI(lifespan=lifespan)

# 环境变量设置（可手动设置）
host = os.environ.get('A2A_UI_HOST', '127.0.0.1')
port = int(os.environ.get('A2A_UI_PORT', '12000'))

host_agent_service.server_url = f"http://{host}:{port}"

# 启动服务
config = uvicorn.Config(app=app, host=host, port=port, timeout_graceful_shutdown=0)
server = uvicorn.Server(config)

import threading
def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")
threading.Thread(target=run_uvicorn, daemon=True).start()



In [ ]:
# ===== 1. 基本依赖 =====
import os, uvicorn, threading
from fastapi import FastAPI

# ===== 2. 创建 FastAPI 应用=====
app = FastAPI(lifespan=lifespan)

# ===== 3. 读取或设定端口 =====
host = os.environ.get('A2A_UI_HOST', '127.0.0.1')      # 默认 127.0.0.1
port = int(os.environ.get('A2A_UI_PORT', '12000'))     # 默认 12000

host_agent_service.server_url = f"http://{host}:{port}"

# ===== 4. 后台线程启动 uvicorn，并打印提示 =====
def run_uvicorn():
    """
    在后台线程启动 uvicorn：
   
    • log_level='info' 让启动 banner 打印出来
    """
    uvicorn.run(app, host=host, port=port, log_level="info")

threading.Thread(target=run_uvicorn, daemon=True).start()

print(f"✅ Uvicorn 已在后台启动： http://{host}:{port}  （浏览器打开这个地址试试）")


✅ Uvicorn 已在后台启动： http://127.0.0.1:12000  （浏览器打开这个地址试试）


INFO:     Started server process [11012]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     127.0.0.1:5205 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:5205 - "GET /styles.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:5204 - "GET /zone.js/bundles/zone.umd.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:5214 - "GET /prod_bundle.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:5214 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:5214 - "GET /__web-components-module__/components/async_poller.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:5214 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:5214 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:5311 - "POST /conversation/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5314 - "POST /task/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5332 - "POST /message/pending HTTP/1.1" 200 OK
INFO:     127.0.0.1:5214 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:5342 - "POST /conversation/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5344 - "POST /task/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5346 - "POST /message/pending HTTP/1.1" 200 OK
INFO:     12